## Практическая работа №10
## Выделение ключевых слов

### Решение:
1. Загрузка данных
Загрузить файл с помощью библиотеки для чтения pdf файлов
(PyPDF2, PDFMiner и пр.). Для разделения текста на отдельные статьи
можно опираться на УДК (использовать в качестве разделителя) или
распарсить оглавление и каждую статью определять в диапазоне
страниц из этого оглавления.

In [1]:
%pip install PyPDF2
%pip install ipywidgets
%pip install rake_nltk
%pip install yake
%pip install summa
%pip install stanza
%pip install jupyter
%pip install scikit-learn

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviou

In [2]:
import PyPDF2

data = []

with open('./yazik/yazik_lab10/data/Математическое и информационное моделирование_2020 (1).pdf', 'rb') as f:
    pdf = PyPDF2.PdfReader(f)
    for page in pdf.pages:
        text = page.extract_text()
        data.append(text)

In [11]:
import pandas as pd

articles = []
article = []
key_words = []
for page in data:
    if 'УДК' in page:
        article = ' '.join(article)
        articles.append(article)
        article = []
    article.append(page)

# Remove first two empty articles
articles = articles[2:]

key_words_list = []

for article in articles:
    key_words = []
    lines = article.split('\n')
    
    for line in lines:
        #if '.' in line:
        #    break
            
        if 'Ключевые слова' in line:
            # Iterate through current and next lines until not find dot
            for i in range(lines.index(line), len(lines)):
                if '.' in lines[i]:
                    key_words.append(lines[i])
                    break
                key_words.append(lines[i])
            break
    # Объедини в строку и раздели по запятой
    key_words = ', '.join(key_words)
    key_words_list.append(key_words)

df = pd.DataFrame({"text":articles, "key_words":key_words_list})

In [12]:
print(df['key_words'][0])

Ключевые слова:  математическое  моделирование, специ ализированная , программа  MDC -SPICE ,  кодирование информации в задержки и в частоту , импульсов,  виртуальный входной нейрон, популяция нейронов.  


In [14]:
import nltk

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

import stanza
stanza.download("ru")


def preprocess(text, stz):
    from nltk.tokenize import sent_tokenize
    from string import punctuation
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords

    sentences = sent_tokenize(text)
    # Lower case
    sentences = [sentence.lower() for sentence in sentences]
    # Remove punctuation
    sentences = [
        "".join(c for c in sentence if c not in punctuation) for sentence in sentences
    ]
    # Tokenize words
    sentences = [word_tokenize(sentence) for sentence in sentences]
    # Remove stopwords
    stop_words = stopwords.words("russian")
    sentences = [
        [word for word in sentence if word not in stop_words] for sentence in sentences
    ]
    # Remove links
    sentences = [
        [word for word in sentence if not word.startswith("http")]
        for sentence in sentences
    ]
    # Remove words with length less than 3
    sentences = [
        [word for word in sentence if len(word) >= 3] for sentence in sentences
    ]
    # Lemmatize
    sentences = [
        [stz(word).to_dict()[0][0]['lemma'] for word in sentence] for sentence in sentences
    ]
    
    words = " ".join([" ".join(sentence) for sentence in sentences])
    return words

[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vscode/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


2023-11-23 17:32:09 INFO: Downloading default packages for language: ru (Russian) ...


2023-11-23 17:32:18 INFO: Finished downloading models and saved to /home/vscode/stanza_resources.


In [15]:
# Preprocess text

nlp = stanza.Pipeline(
    lang="ru",
    processors="tokenize,lemma",
)

2023-11-23 17:34:22 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-23 17:34:23 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| lemma     | syntagrus_nocharlm |

2023-11-23 17:34:23 INFO: Using device: cpu


2023-11-23 17:34:23 INFO: Loading: tokenize
2023-11-23 17:34:23 INFO: Loading: lemma
2023-11-23 17:34:23 INFO: Done loading processors!


In [16]:
from tqdm import tqdm

tqdm.pandas()

df["text_processed"] = df["text"].progress_apply(preprocess, args=(nlp,))

100%|██████████| 85/85 [03:24<00:00,  2.41s/it]


# TFIDF

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

In [18]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df["text_processed"].values)
tfidf_matrix_not_processed = tfidf_vectorizer.fit_transform(df["text"].values)

In [19]:
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.toarray()[41]
feature_names_not_processed = tfidf_vectorizer.get_feature_names_out()
tfidf_scores_not_processed = tfidf_matrix_not_processed.toarray()[41]

In [20]:
sorted_keywords = [
    word for _, word in sorted(zip(tfidf_scores, feature_names), reverse=True)
]
sorted_keywords_not_processed = [
    word
    for _, word in sorted(
        zip(tfidf_scores_not_processed, feature_names_not_processed), reverse=True
    )
]



print("Ключевые слова:", sorted_keywords[:10])
print("Ключевые слова в необработанном df", sorted_keywords_not_processed[:10])
print("Ключевые слова в df['key_words']", df['key_words'][41])

Ключевые слова: ['определенную', 'виртуальных', 'wu', 'етствии', 'выражать', 'каталога', 'диапазон', 'корпус', 'определения', 'адекватных']
Ключевые слова в необработанном df ['шифров', 'xlps', 'замены', 'группы', 'шифры', 'перестановки', 'групп', 'коммутативность', 'шифрования', 'представление']
Ключевые слова в df['key_words'] Ключевые слова. Группы,  представления групп, коммутативность, XLPS -


# RAKE

In [76]:
from rake_nltk import Metric, Rake

PAGE = 41

non_processed_articels = df["text"].values

best_res = []
best_sum = 0
best_index = 0
for i in range(len(non_processed_articels)):
    r = Rake(max_length=5, ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO, min_length=1)
    r.extract_keywords_from_text(non_processed_articels[i])
    res = r.get_ranked_phrases_with_scores()[:5]
    sum = 0
    for i in range(len(res)):
        sum += res[i][0]
    if sum > best_sum:
        best_sum = sum
        best_res = res
        best_index = i

processed_articels = df["text_processed"].values
best_res_processed = []
best_sum_processed = 0
best_index_processed = 0
for i in range(len(processed_articels)):
    r = Rake(max_length=5, ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO, min_length=1)
    r.extract_keywords_from_text(processed_articels[i])
    res = r.get_ranked_phrases_with_scores()[:5]
    sum = 0
    for i in range(len(res)):
        sum += res[i][0]
    if sum > best_sum_processed:
        best_sum_processed = sum
        best_res_processed = res
        best_index_processed = i

print("Ключевые слова:", best_res)
print("Ключевые слова в df['key_words']", df['key_words'][best_index])
print()
print("Ключевые слова в процессированном df", best_res_processed)
print("Ключевые слова в df['key_words']", df['key_words'][best_index_processed])



Ключевые слова: [(25.0, 'существует несколько способов кодирования информации'), (25.0, 'соответствующими режиму этого компоне нта'), (25.0, 'предназначенную для расчета больших схем'), (25.0, 'по умолчанию используется метод трапеций'), (25.0, 'который минимизирует взаимовлияние соседних ячеек')]
Ключевые слова в df['key_words'] Ключевые слова:  распознавание эмоций, методы ИАД, вейвлеты, , распознавание обр азов.  

Ключевые слова в процессированном df [(25.0, 'faster secure computation acm computer'), (16.0, 'covert adversaries efficient protocols'), (16.0, 'complexity private computations symposiчmий'), (15.0, 'universally composиble obliviousый transfer'), (10.0, 'ficient obliviousый transfer')]
Ключевые слова в df['key_words'] Ключевые слова:  распознавание эмоций, методы ИАД, вейвлеты, , распознавание обр азов.  


В данном случае происходит выделение фраз на основе стоп-слов, поэтому необходимость в предобработке отсутствует

# YAKE

In [102]:
import yake

language = "ru"
max_ngram_size = 3
deduplication_threshold = 0.9
deduplication_algo = "seqm"
windowSize = 10
numOfKeywords = 20

custom_kw_extractor = yake.KeywordExtractor(
    lan=language,
    n=max_ngram_size,
    dedupLim=deduplication_threshold,
    dedupFunc=deduplication_algo,
    windowsSize=windowSize,
    top=numOfKeywords,
    features=None,
)
keywords = custom_kw_extractor.extract_keywords(non_processed_articels[44])
keywords_proc = custom_kw_extractor.extract_keywords(df["text_processed"].values[44])
# Sort by score
keywords = sorted(keywords, key=lambda x: x[1], reverse=True)
keywords_proc = sorted(keywords_proc, key=lambda x: x[1], reverse=True)

for i, kw in enumerate(keywords):
    print(kw)
    if i == 10:
        break

print("Предобработанные данные")

for i, kw in enumerate(keywords_proc):
    print(kw)
    if i == 10:
        break

for i, kw in enumerate(df['key_words'][44].split(',')):
    print(kw)
    if i == 10:
        break

('ПРОБЛЕМЫ ПОСТРОЕНИЯ ВИРТУ', 0.033651159653327656)
('VMWARE', 0.03364092562021788)
('Client', 0.02997498890429375)
('виртуальных машин', 0.029759374862845068)
('виртуальных', 0.02974860468492308)
('Windows', 0.0223315918982741)
('ЛАБОР АТОРИИ VIPNET', 0.019733625624804303)
('Coordinator', 0.01756764186623393)
('ViPNet Client', 0.013329582969097045)
('ПОСТРОЕНИЯ ВИРТУ АЛЬНОЙ', 0.011209243284611562)
('ВИРТУ АЛЬНОЙ ЛАБОР', 0.011209243284611562)
Предобработанные данные
('виртуальный машина', 8.929589743169893e-05)
('vipnet coordinator vipnet', 8.509606992795346e-05)
('виртуальный машина необходимый', 8.382097237471218e-05)
('vipnet client перезагрузокать', 8.356367264799744e-05)
('работа курс vipnet', 8.220714800110118e-05)
('таблица таблица характеристика', 7.928760919793047e-05)
('колет ядро процессор', 7.858313237280818e-05)
('ядро процессор колет', 7.858313237280818e-05)
('установка vipnet client', 7.34458267956535e-05)
('процессор windows server', 7.258532729634169e-05)
('vipnet база

Имеет наибольшую схожесть с указанными в тексте ключевыми словами